In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import eli5
import shap
from IPython.display import HTML
import json
import altair as alt

import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
## Zaman degiskenleri icin gerekli kutuphaneleri yukledik
import os
import time
import datetime
import json
import gc
from numba import jit

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn import metrics

from itertools import product

import altair as alt
from altair.vega import v5
from IPython.display import HTML

# using ideas from this kernel: https://www.kaggle.com/notslush/altair-visualization-2018-stackoverflow-survey
def prepare_altair():
    """
    Helper function to prepare altair for working.
    """

    vega_url = 'https://cdn.jsdelivr.net/npm/vega@' + v5.SCHEMA_VERSION
    vega_lib_url = 'https://cdn.jsdelivr.net/npm/vega-lib'
    vega_lite_url = 'https://cdn.jsdelivr.net/npm/vega-lite@' + alt.SCHEMA_VERSION
    vega_embed_url = 'https://cdn.jsdelivr.net/npm/vega-embed@3'
    noext = "?noext"
    
    paths = {
        'vega': vega_url + noext,
        'vega-lib': vega_lib_url + noext,
        'vega-lite': vega_lite_url + noext,
        'vega-embed': vega_embed_url + noext
    }
    
    workaround = f"""    requirejs.config({{
        baseUrl: 'https://cdn.jsdelivr.net/npm/',
        paths: {paths}
    }});
    """
    
    return workaround
    

def add_autoincrement(render_func):
    # Keep track of unique <div/> IDs
    cache = {}
    def wrapped(chart, id="vega-chart", autoincrement=True):
        if autoincrement:
            if id in cache:
                counter = 1 + cache[id]
                cache[id] = counter
            else:
                cache[id] = 0
            actual_id = id if cache[id] == 0 else id + '-' + str(cache[id])
        else:
            if id not in cache:
                cache[id] = 0
            actual_id = id
        return render_func(chart, id=actual_id)
    # Cache will stay outside and 
    return wrapped
           

@add_autoincrement
def render(chart, id="vega-chart"):
    """
    Helper function to plot altair visualizations.
    """
    chart_str = """
    <div id="{id}"></div><script>
    require(["vega-embed"], function(vg_embed) {{
        const spec = {chart};     
        vg_embed("#{id}", spec, {{defaultStyle: true}}).catch(console.warn);
        console.log("anything?");
    }});
    console.log("really...anything?");
    </script>
    """
    return HTML(
        chart_str.format(
            id=id,
            chart=json.dumps(chart) if isinstance(chart, dict) else chart.to_json(indent=None)
        )
    )
    

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                c_prec = df[col].apply(lambda x: np.finfo(x).precision).max()
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max and c_prec == np.finfo(np.float32).precision:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
    

@jit
def fast_auc(y_true, y_prob):
    """
    fast roc_auc computation: https://www.kaggle.com/c/microsoft-malware-prediction/discussion/76013
    """
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    nfalse = 0
    auc = 0
    n = len(y_true)
    for i in range(n):
        y_i = y_true[i]
        nfalse += (1 - y_i)
        auc += y_i * nfalse
    auc /= (nfalse * (n - nfalse))
    return auc


def eval_auc(y_true, y_pred):
    """
    Fast auc eval function for lgb.
    """
    return 'auc', fast_auc(y_true, y_pred), True


def group_mean_log_mae(y_true, y_pred, types, floor=1e-9):
    """
    Fast metric computation for this competition: https://www.kaggle.com/c/champs-scalar-coupling
    Code is from this kernel: https://www.kaggle.com/uberkinder/efficient-metric
    """
    maes = (y_true-y_pred).abs().groupby(types).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()
    

def train_model_regression(X, X_test, y, params, folds=None, model_type='lgb', eval_metric='mae', columns=None, plot_feature_importance=False, model=None,
                               verbose=10000, early_stopping_rounds=200, n_estimators=50000, splits=None, n_folds=3):
    """
    A function to train a variety of regression models.
    Returns dictionary with oof predictions, test predictions, scores and, if necessary, feature importances.
    
    :params: X - training data, can be pd.DataFrame or np.ndarray (after normalizing)
    :params: X_test - test data, can be pd.DataFrame or np.ndarray (after normalizing)
    :params: y - target
    :params: folds - folds to split data
    :params: model_type - type of model to use
    :params: eval_metric - metric to use
    :params: columns - columns to use. If None - use all columns
    :params: plot_feature_importance - whether to plot feature importance of LGB
    :params: model - sklearn model, works only for "sklearn" model type
    
    """
    columns = X.columns if columns is None else columns
    X_test = X_test[columns]
    splits = folds.split(X) if splits is None else splits
    n_splits = folds.n_splits if splits is None else n_folds
    
    # to set up scoring parameters
    metrics_dict = {'mae': {'lgb_metric_name': 'mae',
                        'catboost_metric_name': 'MAE',
                        'sklearn_scoring_function': metrics.mean_absolute_error},
                    'group_mae': {'lgb_metric_name': 'mae',
                        'catboost_metric_name': 'MAE',
                        'scoring_function': group_mean_log_mae},
                    'mse': {'lgb_metric_name': 'mse',
                        'catboost_metric_name': 'MSE',
                        'sklearn_scoring_function': metrics.mean_squared_error}
                    }

    
    result_dict = {}
    
    # out-of-fold predictions on train data
    oof = np.zeros(len(X))
    
    # averaged predictions on train data
    prediction = np.zeros(len(X_test))
    
    # list of scores on folds
    scores = []
    feature_importance = pd.DataFrame()
    
    # split and train on folds
    for fold_n, (train_index, valid_index) in enumerate(splits):
        if verbose:
            print(f'Fold {fold_n + 1} started at {time.ctime()}')
        if type(X) == np.ndarray:
            X_train, X_valid = X[columns][train_index], X[columns][valid_index]
            y_train, y_valid = y[train_index], y[valid_index]
        else:
            X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
            
        if model_type == 'lgb':
            model = lgb.LGBMRegressor(**params, n_estimators = n_estimators, n_jobs = -1)
            model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric=metrics_dict[eval_metric]['lgb_metric_name'],
                    verbose=verbose, early_stopping_rounds=early_stopping_rounds)
            
            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X.columns)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X.columns)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=verbose, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
        
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            score = metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid)
            print(f'Fold {fold_n}. {eval_metric}: {score:.4f}.')
            print('')
            
            y_pred = model.predict(X_test).reshape(-1,)
        
        if model_type == 'cat':
            model = CatBoostRegressor(iterations=20000,  eval_metric=metrics_dict[eval_metric]['catboost_metric_name'], **params,
                                      loss_function=metrics_dict[eval_metric]['catboost_metric_name'])
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[], use_best_model=True, verbose=False)

            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test)
        
        oof[valid_index] = y_pred_valid.reshape(-1,)
        if eval_metric != 'group_mae':
            scores.append(metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid))
        else:
            scores.append(metrics_dict[eval_metric]['scoring_function'](y_valid, y_pred_valid, X_valid['type']))

        prediction += y_pred    
        
        if model_type == 'lgb' and plot_feature_importance:
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = columns
            fold_importance["importance"] = model.feature_importances_
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_splits
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    result_dict['oof'] = oof
    result_dict['prediction'] = prediction
    result_dict['scores'] = scores
    
    if model_type == 'lgb':
        if plot_feature_importance:
            feature_importance["importance"] /= n_splits
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12));
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
            plt.title('LGB Features (avg over folds)');
            
            result_dict['feature_importance'] = feature_importance
        
    return result_dict
    


def train_model_classification(X, X_test, y, params, folds, model_type='lgb', eval_metric='auc', columns=None, plot_feature_importance=False, model=None,
                               verbose=10000, early_stopping_rounds=200, n_estimators=50000, splits=None, n_folds=3, averaging='usual', n_jobs=-1):
    """
    A function to train a variety of classification models.
    Returns dictionary with oof predictions, test predictions, scores and, if necessary, feature importances.
    
    :params: X - training data, can be pd.DataFrame or np.ndarray (after normalizing)
    :params: X_test - test data, can be pd.DataFrame or np.ndarray (after normalizing)
    :params: y - target
    :params: folds - folds to split data
    :params: model_type - type of model to use
    :params: eval_metric - metric to use
    :params: columns - columns to use. If None - use all columns
    :params: plot_feature_importance - whether to plot feature importance of LGB
    :params: model - sklearn model, works only for "sklearn" model type
    
    """
    columns = X.columns if columns is None else columns
    n_splits = folds.n_splits if splits is None else n_folds
    X_test = X_test[columns]
    
    # to set up scoring parameters
    metrics_dict = {'auc': {'lgb_metric_name': eval_auc,
                        'catboost_metric_name': 'AUC',
                        'sklearn_scoring_function': metrics.roc_auc_score},
                    }
    
    result_dict = {}
    if averaging == 'usual':
        # out-of-fold predictions on train data
        oof = np.zeros((len(X), 1))

        # averaged predictions on train data
        prediction = np.zeros((len(X_test), 1))
        
    elif averaging == 'rank':
        # out-of-fold predictions on train data
        oof = np.zeros((len(X), 1))

        # averaged predictions on train data
        prediction = np.zeros((len(X_test), 1))

    
    # list of scores on folds
    scores = []
    feature_importance = pd.DataFrame()
    
    # split and train on folds
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        print(f'Fold {fold_n + 1} started at {time.ctime()}')
        if type(X) == np.ndarray:
            X_train, X_valid = X[columns][train_index], X[columns][valid_index]
            y_train, y_valid = y[train_index], y[valid_index]
        else:
            X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
            
        if model_type == 'lgb':
            model = lgb.LGBMClassifier(**params, n_estimators=n_estimators, n_jobs = n_jobs)
            model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric=metrics_dict[eval_metric]['lgb_metric_name'],
                    verbose=verbose, early_stopping_rounds=early_stopping_rounds)
            
            y_pred_valid = model.predict_proba(X_valid)[:, 1]
            y_pred = model.predict_proba(X_test, num_iteration=model.best_iteration_)[:, 1]
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X.columns)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X.columns)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=n_estimators, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=verbose, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
        
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            score = metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid)
            print(f'Fold {fold_n}. {eval_metric}: {score:.4f}.')
            print('')
            
            y_pred = model.predict_proba(X_test)
        
        if model_type == 'cat':
            model = CatBoostClassifier(iterations=n_estimators, eval_metric=metrics_dict[eval_metric]['catboost_metric_name'], **params,
                                      loss_function=Logloss)
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[], use_best_model=True, verbose=False)

            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test)
        
        if averaging == 'usual':
            
            oof[valid_index] = y_pred_valid.reshape(-1, 1)
            scores.append(metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid))
            
            prediction += y_pred.reshape(-1, 1)

        elif averaging == 'rank':
                                  
            oof[valid_index] = y_pred_valid.reshape(-1, 1)
            scores.append(metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid))
                                  
            prediction += pd.Series(y_pred).rank().values.reshape(-1, 1)        
        
        if model_type == 'lgb' and plot_feature_importance:
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = columns
            fold_importance["importance"] = model.feature_importances_
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_splits
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    result_dict['oof'] = oof
    result_dict['prediction'] = prediction
    result_dict['scores'] = scores
    
    if model_type == 'lgb':
        if plot_feature_importance:
            feature_importance["importance"] /= n_splits
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12));
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
            plt.title('LGB Features (avg over folds)');
            
            result_dict['feature_importance'] = feature_importance
            result_dict['top_columns'] = cols
        
    return result_dict

# setting up altair
workaround = prepare_altair()
HTML("".join((
    "<script>",
    workaround,
    "</script>",
)))

In [ ]:
# General imports
import numpy as np
import pandas as pd
import os, warnings, datetime, math
import pandas as pd
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.width', 200)
pd.set_option('display.max_rows', 500)


In [ ]:
# datalarin yuklenmesi
train_transaction = pd.read_csv("../input/ieee-fraud-detection/train_transaction.csv")
test_transaction  = pd.read_csv("../input/ieee-fraud-detection/test_transaction.csv")

test_identity = pd.read_csv("../input/ieee-fraud-detection/test_identity.csv")
train_identity = pd.read_csv("../input/ieee-fraud-detection/train_identity.csv")


In [ ]:
#kullanilan memory azaltmak icin fonksiyon tanimladik
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [ ]:
# kolon isimlerini esitliyoruz, alt ve orta cizgi farki var
test_identity.columns=train_identity.columns


In [ ]:
# trainleri birleştirdik:
train = pd.merge(train_transaction,train_identity, on="TransactionID", how="left")
train.head()

In [ ]:
# testleri birleştirdik:
test = pd.merge(test_transaction,test_identity, on="TransactionID", how="left")
test.head()

In [ ]:
# test ve train datasetinin memory kullanimini azalttik
train = reduce_mem_usage(train)
train.head()
test = reduce_mem_usage(test)

In [ ]:
#kullanmayacagimizi siliyoruz
del train_identity, train_transaction, test_identity, test_transaction

In [ ]:
#Feature selection icin daha sonra kullanacagiz
test_columns=test.columns
train_columns=train.columns

In [ ]:
one_value_cols = [col for col in train.columns if train[col].nunique() <= 1]
one_value_cols_test = [col for col in test.columns if test[col].nunique() <= 1]
one_value_cols == one_value_cols_test

FEATURE

In [ ]:
# SUNUMDA GÖSTER!!!!

# New feature: number of NaN's
#We have plenty of NaN's in this dataset and they can have a significant effect so' \
#' why don't we use them? I am adding a new column to the dateset, which will contain
#"a number of NaN for each row. So if a row
#(a single training example) contain, say, 10 NaNs, a new feature's value for this row will be 10.

# rowlardaki na'ları sayıyor ve fe olarak tutuyor.
train['nulls'] = train.isnull().sum(axis=1)
test['nulls'] = test.isnull().sum(axis=1)

In [ ]:
train['nulls'][0:5]

In [ ]:
## GÖSTER!!

#############card2#########################
#card2 in frekanslarini encode edip yeni bir feature urettik

temp = pd.concat([train['card2'], test['card2']], ignore_index=True).value_counts(dropna=False)
train['card2_count'] = train['card2'].map(temp)
test['card2_count'] = test['card2'].map(temp)
del temp


In [ ]:
train['card2_count'][0:5]

In [ ]:
## GÖSTER!!!!

#####################TransactionAmount############
# Transaction amount'un virgulden sonraki kismiyla yeni bir feature urettik, fraudlarda virgulden sonrasi daha fazla
#Decimal part of transaction amount.
#Lenght of the decimal part of transaction amount. What does it mean?
# Well, if lenght is 1 or 2 signs it is totaly understandable - it might be cents.
# But what is wrong with a decimal part's lenght being 3 and more sings? Maybe it is due to a currency convertion?

train['TransactionAmt_decimal'] = ((train['TransactionAmt'] - train['TransactionAmt'].astype(int)) * 1000).astype(int)
test['TransactionAmt_decimal'] = ((test['TransactionAmt'] - test['TransactionAmt'].astype(int)) * 1000).astype(int)
# plot_numerical('TransactionAmt_decimal')

In [ ]:
train['TransactionAmt_decimal'].value_counts()

In [ ]:
#virgulden sonraki uzunluk bir feature olarak yaratildi
train['TransactionAmt_decimal_lenght'] = train['TransactionAmt'].astype(str).str.split('.', expand=True)[1].str.len()
test['TransactionAmt_decimal_lenght'] = test['TransactionAmt'].astype(str).str.split('.', expand=True)[1].str.len()

In [ ]:
train['TransactionAmt_decimal'][0:5]

In [ ]:
train['TransactionAmt_decimal_lenght'].value_counts()

In [ ]:
# https://www.kaggle.com/artgor/eda-and-models

##card1 >> card1 could be the Issuer Identification Number (IIN), encoded as a categorical variable with LabelEncoder.
## card4 >> mastercard visa

# >> card1 ve  card4 e gore 'TransactionAmt' un mean standart sapma alip, yeni feature urettik, train icin

train['TransactionAmt_to_mean_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_mean_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_std_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('std')
train['TransactionAmt_to_std_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('std')

In [ ]:
##  ayni islem test icin
test['TransactionAmt_to_mean_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_mean_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_std_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('std')
test['TransactionAmt_to_std_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('std')


In [ ]:
## card1 ve  card4 e gore gore id_02 ye gore group by alip, mean standart sapmasini aldik
#id_02 sanirim balance bilgisi
train['id_02_to_mean_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('mean')
train['id_02_to_mean_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('mean')
train['id_02_to_std_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('std')
train['id_02_to_std_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('std')

In [ ]:
# ayni islemi test datasina da gerceklestirdik
test['id_02_to_mean_card1'] = test['id_02'] / test.groupby(['card1'])['id_02'].transform('mean')
test['id_02_to_mean_card4'] = test['id_02'] / test.groupby(['card4'])['id_02'].transform('mean')
test['id_02_to_std_card1'] = test['id_02'] / test.groupby(['card1'])['id_02'].transform('std')
test['id_02_to_std_card4'] = test['id_02'] / test.groupby(['card4'])['id_02'].transform('std')


In [ ]:
## D li featurelar zamana ait featurelar, zaman odaginda, group by card1 ve card4 alinarak
## mean ve std ler feature haline getirildi

train['D15_to_mean_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('mean')
train['D15_to_mean_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('mean')
train['D15_to_std_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('std')
train['D15_to_std_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('std')

test['D15_to_mean_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('mean')
test['D15_to_mean_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('mean')
test['D15_to_std_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('std')
test['D15_to_std_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('std')

train['D15_to_mean_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('mean')
train['D15_to_mean_addr2'] = train['D15'] / train.groupby(['addr2'])['D15'].transform('mean')
train['D15_to_std_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('std')
train['D15_to_std_addr2'] = train['D15'] / train.groupby(['addr2'])['D15'].transform('std')

test['D15_to_mean_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('mean')
test['D15_to_mean_addr2'] = test['D15'] / test.groupby(['addr2'])['D15'].transform('mean')
test['D15_to_std_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('std')
test['D15_to_std_addr2'] = test['D15'] / test.groupby(['addr2'])['D15'].transform('std')


In [ ]:
# email,noktaya gore 3 parcaya bolundu, test ve train icin
train[['P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3']] = train['P_emaildomain'].str.split('.', expand=True)
train[['R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']] = train['R_emaildomain'].str.split('.', expand=True)
test[['P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3']] = test['P_emaildomain'].str.split('.', expand=True)
test[['R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']] = test['R_emaildomain'].str.split('.', expand=True)

In [ ]:
## Tarayici guncelleyenleri yakalayabilecegimiz bir feature urettik

#https://www.kaggle.com/yasagure/fraud-makers-may-be-earnest-people-about-browser?scriptVersionId=21010669
## This result suggests that people who update browser are more possibly fraud makers than people who are lazy about browser.
## Fraud makers may be earnest people.

## #https://en.wikipedia.org/wiki/List_of_Microsoft_Windows_versions(2019/8/10)
## #windows 10 July 29, 2015
## #windows 8.1 October 17, 2013
## #Windows 8 October 26, 2012
## #Windows 7 October 22, 2009
## #windows vista November 8, 2006
## #windows XP October 25, 2001


## iOS
## https://en.wikipedia.org/wiki/IOS_version_history
##
## 12.4 July 22, 2019
## 10.3.4 July 22, 2019
## 10.3.3 July 19, 2017
## 9.3.6 July 22, 2019
## 9.3.5 August 25, 2016
## 7.1.2 June 30, 2014
## 6.1.6 February 21, 2014
## 5.1.1 May 7, 2012
## 4.2.1 November 22, 2010
## 3.1.3 February 2, 2010

In [ ]:
a = np.zeros(train.shape[0])
a = np.nan
train["lastest_browser"] = a

train.lastest_browser[train["id_31"]=="samsung browser 7.0"]=1
train.lastest_browser[train["id_31"]=="opera 53.0"]=1
train.lastest_browser[train["id_31"]=="mobile safari 10.0"]=1
train.lastest_browser[train["id_31"]=="google search application 49.0"]=1
train.lastest_browser[train["id_31"]=="firefox 60.0"]=1
train.lastest_browser[train["id_31"]=="edge 17.0"]=1
train.lastest_browser[train["id_31"]=="chrome 69.0"]=1
train.lastest_browser[train["id_31"]=="chrome 67.0 for android"]=1
train.lastest_browser[train["id_31"]=="chrome 63.0"]=1
train.lastest_browser[train["id_31"]=="chrome 63.0 for android"]=1
train.lastest_browser[train["id_31"]=="chrome 63.0 for ios"]=1
train.lastest_browser[train["id_31"]=="chrome 64.0"]=1
train.lastest_browser[train["id_31"]=="chrome 64.0 for android"]=1
train.lastest_browser[train["id_31"]=="chrome 64.0 for ios"]=1
train.lastest_browser[train["id_31"]=="chrome 65.0"]=1
train.lastest_browser[train["id_31"]=="chrome 65.0 for android"]=1
train.lastest_browser[train["id_31"]=="chrome 65.0 for ios"]=1
train.lastest_browser[train["id_31"]=="chrome 66.0"]=1
train.lastest_browser[train["id_31"]=="chrome 66.0 for android"]=1
train.lastest_browser[train["id_31"]=="chrome 66.0 for ios"]=1

In [ ]:
train.lastest_browser.head()

In [ ]:
train.groupby("lastest_browser").mean()["isFraud"]

In [ ]:
#aynisini test icin yaparsak:

b = np.zeros(test.shape[0])
b = np.nan
test["lastest_browser"] = b

test.lastest_browser[test["id_31"]=="samsung browser 7.0"]=1
test.lastest_browser[test["id_31"]=="opera 53.0"]=1
test.lastest_browser[test["id_31"]=="mobile safari 10.0"]=1
test.lastest_browser[test["id_31"]=="google search application 49.0"]=1
test.lastest_browser[test["id_31"]=="firefox 60.0"]=1
test.lastest_browser[test["id_31"]=="edge 17.0"]=1
test.lastest_browser[test["id_31"]=="chrome 69.0"]=1
test.lastest_browser[test["id_31"]=="chrome 67.0 for android"]=1
test.lastest_browser[test["id_31"]=="chrome 63.0"]=1
test.lastest_browser[test["id_31"]=="chrome 63.0 for android"]=1
test.lastest_browser[test["id_31"]=="chrome 63.0 for ios"]=1
test.lastest_browser[test["id_31"]=="chrome 64.0"]=1
test.lastest_browser[test["id_31"]=="chrome 64.0 for android"]=1
test.lastest_browser[test["id_31"]=="chrome 64.0 for ios"]=1
test.lastest_browser[test["id_31"]=="chrome 65.0"]=1
test.lastest_browser[test["id_31"]=="chrome 65.0 for android"]=1
test.lastest_browser[test["id_31"]=="chrome 65.0 for ios"]=1
test.lastest_browser[test["id_31"]=="chrome 66.0"]=1
test.lastest_browser[test["id_31"]=="chrome 66.0 for android"]=1
test.lastest_browser[test["id_31"]=="chrome 66.0 for ios"]=1

In [ ]:
############Tarayici#####################
# tarayicilari siniflandirdik
#There are a lot of categories in id_31.
##here, I change bool data to numeric data by multipling 1( * 1)

train["chrome"]=train['id_31'].str.contains('chrome')*1
train["samsung_browser"] = train['id_31'].str.contains('samsung')*1
train["opera"] = train['id_31'].str.contains('opera')*1
train["ie"] = train['id_31'].str.contains('ie')*1
train["google_browser"] = train['id_31'].str.contains('google')*1
train["firefox"] = train['id_31'].str.contains('firefox')*1
train["edge"] = train['id_31'].str.contains('edge')*1

# when it comes to Android browser, I will use different way because chrome has "chrome for android"

train["android_browser"] = train['id_31'].str.contains('android browser')*1
train["android_browser"] = train['id_31'].str.contains('android webview')*1
train["android_browser"] = train['id_31'].str.contains('Generic/Android')*1
train["android_browser"] = train['id_31'].str.contains('Generic/Android 7.0')*1


In [ ]:
# chrome browser user is more possibly fraud maker than people who do not use it.
# This may be because chrome has a lot of extension
train.groupby("chrome").mean().isFraud

In [ ]:
#aynisini test icin yaptik


test["chrome"]=test['id_31'].str.contains('chrome')*1
test["samsung_browser"] = test['id_31'].str.contains('samsung')*1
test["opera"] = test['id_31'].str.contains('opera')*1
test["ie"] = test['id_31'].str.contains('ie')*1
test["google_browser"] = test['id_31'].str.contains('google')*1
test["firefox"] = test['id_31'].str.contains('firefox')*1
test["edge"] = test['id_31'].str.contains('edge')*1

# when it comes to Android browser, I will use different way because chrome has "chrome for android"

test["android_browser"] = train['id_31'].str.contains('android browser')*1
test["android_browser"] = train['id_31'].str.contains('android webview')*1
test["android_browser"] = train['id_31'].str.contains('Generic/Android')*1
test["android_browser"] = train['id_31'].str.contains('Generic/Android 7.0')*1


In [ ]:
#id_01 i null olanlardan feature urettik
train["null"]=train.id_01.isnull()
train.groupby("null").mean()["isFraud"]

In [ ]:
#aynisini test icin yaptik
test["null"]=test.id_01.isnull()


In [ ]:
##########################################
#Deviceinfo
########################################

#Huawei has a lot of series of device. Blade is also one of them.

def change(some):
    if some != some:
        return np.nan
    elif "Huawei" in some:
        return 1
    elif "Blade" in some:
        return 1

    else:
        return 0

train["Huawei"] = train["DeviceInfo"].map(change)


In [ ]:
train["Huawei"][0:5]

In [ ]:
train.groupby("Huawei").mean().isFraud

In [ ]:
test["ZTE"] = test['DeviceInfo'].str.contains('ZTE')


In [ ]:
train["ZTE"] = train['DeviceInfo'].str.contains('ZTE')

train.groupby("ZTE").mean().isFraud
# ZTE
# False   0.07248
# True    0.18750

In [ ]:
train["iOS Device"] = train['DeviceInfo'].str.contains('iOS Device')*1
train["Lanix"] = train['DeviceInfo'].str.contains('Ilium')*1
train["ASUS"] = train['DeviceInfo'].str.contains('ASUS')*1
train["Hisense"] = train['DeviceInfo'].str.contains('Hisense')*1
train["Desire"] = train['DeviceInfo'].str.contains('Desire')*1
train["LG"] = train['DeviceInfo'].str.contains('LG')*1
train["Alcatel"] = train['DeviceInfo'].str.contains('Alcatel')*1

In [ ]:
## 
test["Huawei"] = test["DeviceInfo"].map(change)

In [ ]:
test["iOS Device"] = test['DeviceInfo'].str.contains('iOS Device')*1
test["Lanix"] = test['DeviceInfo'].str.contains('Ilium')*1
test["ASUS"] = test['DeviceInfo'].str.contains('ASUS')*1
test["Hisense"] = test['DeviceInfo'].str.contains('Hisense')*1
test["Desire"] = test['DeviceInfo'].str.contains('Desire')*1
test["LG"] = test['DeviceInfo'].str.contains('LG')*1
test["Alcatel"] = test['DeviceInfo'].str.contains('Alcatel')*1

In [ ]:
## ve ['TransactionDT'] bu bilgi tutulacak sortlama icin
startdate = datetime.datetime.strptime('2017-11-01', '%Y-%m-%d')
train['TransactionDT_transformed'] = train['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
test['TransactionDT_transformed'] = test['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
train['dow'] = train['TransactionDT_transformed'].dt.dayofweek
train['hour'] = train['TransactionDT_transformed'].dt.hour
test['dow'] = test['TransactionDT_transformed'].dt.dayofweek
test['hour'] = test['TransactionDT_transformed'].dt.hour

In [ ]:
test.shape

In [ ]:
train.shape

In [ ]:
print(test.columns)

In [ ]:
print(train.columns)

In [ ]:
for col in ['card4', 'card6', 'ProductCD']:
    print('Encoding', col)
    temp_df = pd.concat([train[[col]], test[[col]]])
    col_encoded = temp_df[col].value_counts().to_dict()   
    train[col] = train[col].map(col_encoded)
    test[col]  = test[col].map(col_encoded)
    print(col_encoded)

In [ ]:
########################### M columns
#################################################################################
# Converting Strings to ints(or floats if nan in column)

for col in ['M1','M2','M3','M5','M6','M7','M8','M9']:
    train[col] = train[col].map({'T':1, 'F':0})
    test[col]  = test[col].map({'T':1, 'F':0})

for col in ['M4']:
    print('Encoding', col)
    temp_df = pd.concat([train[[col]], test[[col]]])
    col_encoded = temp_df[col].value_counts().to_dict()   
    train[col] = train[col].map(col_encoded)
    test[col]  = test[col].map(col_encoded)
    print(col_encoded)

In [ ]:
########################### Identity columns
#################################################################################

def minify_identity_df(df):

    df['id_12'] = df['id_12'].map({'Found':1, 'NotFound':0})
    df['id_15'] = df['id_15'].map({'New':2, 'Found':1, 'Unknown':0})
    df['id_16'] = df['id_16'].map({'Found':1, 'NotFound':0})

    df['id_23'] = df['id_23'].map({'TRANSPARENT':4, 'IP_PROXY':3, 'IP_PROXY:ANONYMOUS':2, 'IP_PROXY:HIDDEN':1})

    df['id_27'] = df['id_27'].map({'Found':1, 'NotFound':0})
    df['id_28'] = df['id_28'].map({'New':2, 'Found':1})

    df['id_29'] = df['id_29'].map({'Found':1, 'NotFound':0})

    df['id_35'] = df['id_35'].map({'T':1, 'F':0})
    df['id_36'] = df['id_36'].map({'T':1, 'F':0})
    df['id_37'] = df['id_37'].map({'T':1, 'F':0})
    df['id_38'] = df['id_38'].map({'T':1, 'F':0})

    df['id_34'] = df['id_34'].fillna(':0')
    df['id_34'] = df['id_34'].apply(lambda x: x.split(':')[1]).astype(np.int8)
    df['id_34'] = np.where(df['id_34']==0, np.nan, df['id_34'])
    
    df['id_33'] = df['id_33'].fillna('0x0')
    df['id_33_0'] = df['id_33'].apply(lambda x: x.split('x')[0]).astype(int)
    df['id_33_1'] = df['id_33'].apply(lambda x: x.split('x')[1]).astype(int)
    df['id_33'] = np.where(df['id_33']=='0x0', np.nan, df['id_33'])

    df['DeviceType'].map({'desktop':1, 'mobile':0})
    return df

train = minify_identity_df(train)
test = minify_identity_df(test)

for col in ['id_33']:
    train[col] = train[col].fillna('unseen_before_label')
    test[col]  = test[col].fillna('unseen_before_label')
    
    le = LabelEncoder()
    le.fit(list(train[col])+list(test[col]))
    train[col] = le.transform(train[col])
    test[col]  = le.transform(test[col])

In [ ]:
new_train_cols=train.columns
new_test_cols=train.columns


# Preparing For Modelling

In [ ]:
#yeni urettigimiz featurelarin listesi tutalim
new_cols_= [col for col in train.columns if col not in train_columns]
new_cols_

In [ ]:
# yeni uretilen featurelar icerine denk geliyorsa cikarilacak

#cols = [col for col in train.columns if col not in useless_cols]
many_null_cols = [col for col in train_columns if train_columns.isnull().sum() / train.shape[0] > 0.9]
many_null_cols_test = [col for col in test_columns if test[col].isnull().sum() / test.shape[0] > 0.9]

In [ ]:
many_null_cols

In [ ]:
many_null_cols_test

In [ ]:
# yeni uretilen featurelar icerine denk geliyorsa cikarilacak
big_top_value_cols = [col for col in train_columns if train[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
big_top_value_cols_test = [col for col in test_columns if test[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]

In [ ]:
big_top_value_cols

In [ ]:
big_top_value_cols_test

In [ ]:
cols_to_drop = list(set(many_null_cols + many_null_cols_test + big_top_value_cols + big_top_value_cols_test + one_value_cols+ one_value_cols_test))

In [ ]:
cols_to_drop

In [ ]:
cols_to_drop.remove('isFraud')
len(cols_to_drop)

In [ ]:
train = train.drop(cols_to_drop, axis=1)
test = test.drop(cols_to_drop, axis=1)

In [ ]:
train.to_csv('train.csv', index=False)

In [ ]:
test.to_csv('test.csv', index=False)

In [ ]:
cat_cols = ['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29',
            'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo','P_emaildomain',
            'R_emaildomain', 'card1', 'card2', 'card3',  'card5', 'addr1', 'addr2', 'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9',
            'P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3', 'R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']
for col in cat_cols:
    if col in train.columns:
        le = LabelEncoder()
        le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
        test[col] = le.transform(list(test[col].astype(str).values)) 

In [ ]:
#train.to_csv('train.csv', index=True)

#train=pd.read_csv("./train.csv")

In [ ]:
test.head()

In [ ]:
X = train.sort_values('TransactionDT').drop(['isFraud', 'TransactionDT', 'TransactionID','TransactionDT_transformed'], axis=1)
y = train.sort_values('TransactionDT')['isFraud']
#X_test = test.sort_values('TransactionDT').drop(['TransactionDT', 'TransactionID'], axis=1)
X_test = test.drop(['TransactionDT', 'TransactionID','TransactionDT_transformed'], axis=1)
del train
test = test[["TransactionDT", 'TransactionID']]

In [ ]:
X.shape

In [ ]:
X_test.shape

In [ ]:
X=X.loc[0:118108,:]

In [ ]:
X.shape

In [ ]:
y.head()

In [ ]:
y=y.loc[0:118108]

In [ ]:
len(y)

In [ ]:
X["ZTE"].dtypes

In [ ]:
X["ZTE"].value_counts

In [ ]:
X["ZTE"] = X["ZTE"].map({'T':1, 'F':0})

In [ ]:
X["ZTE"].dtypes

In [ ]:
X_test["ZTE"].dtypes

In [ ]:
X_test["ZTE"] = X_test["ZTE"].map({'T':1, 'F':0})

In [ ]:
X_test["ZTE"].dtypes

In [ ]:
# by https://www.kaggle.com/dimartinot
def clean_inf_nan(df):
    return df.replace([np.inf, -np.inf], np.nan)   

# Cleaning infinite values to NaN
X = clean_inf_nan(X)
X_test = clean_inf_nan(X_test)

In [ ]:
n_fold = 5
folds = TimeSeriesSplit(n_splits=n_fold)
folds = KFold(n_splits=5)

In [ ]:
params = {'num_leaves': 256,
          'min_child_samples': 79,
          'objective': 'binary',
          'max_depth': 13,
          'learning_rate': 0.03,
          "boosting_type": "gbdt",
          "subsample_freq": 3,
          "subsample": 0.9,
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3,
          'reg_lambda': 0.3,
          'colsample_bytree': 0.9,
          #'categorical_feature': cat_cols
         }
result_dict_lgb = train_model_classification(X=X, X_test=X_test, y=y, params=params, folds=folds, model_type='lgb', eval_metric='auc', plot_feature_importance=True,
                                                      verbose=500, early_stopping_rounds=200, n_estimators=5000, averaging='usual', n_jobs=-1)

In [ ]:
sub['isFraud'] = result_dict_lgb['prediction']
sub.to_csv('submission.csv', index=False)

In [ ]:
pd.DataFrame(result_dict_lgb['oof']).to_csv('lgb_oof.csv', index=False)